<a href="https://colab.research.google.com/github/Abhi-213/5c-Networks/blob/main/NESTED_UNET_AND_ATTENTION_UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=588a55254d7b96fc1480b45a0b5c39ffc0a46b627183ace247b0922a5892c398
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
import wget


url = "https://dicom5c.blob.core.windows.net/public/Data.zip"

dataset_filename = wget.download(url)


!unzip /content/Data.zip

Streaming output truncated to the last 5000 lines.
  inflating: Data/TCGA_DU_5855_19951217/TCGA_DU_5855_19951217_1_mask.tif  
  inflating: __MACOSX/Data/TCGA_DU_5855_19951217/._TCGA_DU_5855_19951217_1_mask.tif  
  inflating: Data/TCGA_DU_5855_19951217/TCGA_DU_5855_19951217_17_mask.tif  
  inflating: __MACOSX/Data/TCGA_DU_5855_19951217/._TCGA_DU_5855_19951217_17_mask.tif  
  inflating: Data/TCGA_DU_5855_19951217/TCGA_DU_5855_19951217_16_mask.tif  
  inflating: __MACOSX/Data/TCGA_DU_5855_19951217/._TCGA_DU_5855_19951217_16_mask.tif  
  inflating: Data/TCGA_DU_5855_19951217/TCGA_DU_5855_19951217_24_mask.tif  
  inflating: __MACOSX/Data/TCGA_DU_5855_19951217/._TCGA_DU_5855_19951217_24_mask.tif  
  inflating: Data/TCGA_DU_5855_19951217/TCGA_DU_5855_19951217_25_mask.tif  
  inflating: __MACOSX/Data/TCGA_DU_5855_19951217/._TCGA_DU_5855_19951217_25_mask.tif  
  inflating: Data/TCGA_DU_5855_19951217/TCGA_DU_5855_19951217_11.tif  
  inflating: __MACOSX/Data/TCGA_DU_5855_19951217/._TCGA_DU_5855_1

In [ ]:
!pip install albumentations==1.0.3  # For image augmentations
!pip install torch torchvision  # Install PyTorch
!pip install opencv-python-headless  # For OpenCV (headless for Colab)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 25.6 MB/s eta 0:00:00


In [31]:
import os
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Define the CLAHE preprocessing function
def apply_clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

# Custom dataset class for Brain MRI images and masks
class BrainMRIDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load and preprocess the image and mask
        image = cv2.imread(self.image_paths[idx], cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(self.mask_paths[idx], cv2.IMREAD_GRAYSCALE)

        # Apply CLAHE to the image
        image = apply_clahe(image)

        # Normalize the image to [0, 1]
        image = image / 255.0

        # Convert image to float32 for Albumentations compatibility
        image = image.astype(np.float32)

        # Binarize the mask
        mask = np.where(mask > 0, 1, 0).astype(np.float32)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image.float(), mask.unsqueeze(0).float()  # Add channel dimension to mask

# Function to load the images and masks
def load_data(image_dir):
    images = []
    masks = []
    for folder in os.listdir(image_dir):
        folder_path = os.path.join(image_dir, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                if 'mask' in file:
                    masks.append(file_path)
                else:
                    images.append(file_path)
    return images, masks


In [51]:
# Define transformations (augmentation + tensor conversion)
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussianBlur(p=0.2),
    ToTensorV2()
])
val_transform = A.Compose([
    ToTensorV2()
])

# Load images and masks
image_dir = '/content/Data'  # Change this to your actual image directory
images, masks = load_data(image_dir)

# Split dataset into train and validation
train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.25, random_state=42)

train_dataset = BrainMRIDataset(train_images, train_masks, transform=train_transform)
val_dataset = BrainMRIDataset(val_images, val_masks, transform=val_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)  # Increased batch size
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)


In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class NestedUNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(NestedUNet, self).__init__()
        nb_filter = [32, 64, 128, 256, 512]

        # Downsampling path
        self.conv1_1 = ConvBlock(in_channels, nb_filter[0])
        self.conv2_1 = ConvBlock(nb_filter[0], nb_filter[1])
        self.conv3_1 = ConvBlock(nb_filter[1], nb_filter[2])
        self.conv4_1 = ConvBlock(nb_filter[2], nb_filter[3])
        self.conv5_1 = ConvBlock(nb_filter[3], nb_filter[4])

        # Upsampling path
        self.up1_2 = self.up_conv(nb_filter[1], nb_filter[0])
        self.up2_2 = self.up_conv(nb_filter[2], nb_filter[1])
        self.up3_2 = self.up_conv(nb_filter[3], nb_filter[2])
        self.up4_2 = self.up_conv(nb_filter[4], nb_filter[3])

        # Define the ConvBlocks for the upsampling path
        self.conv1_2 = ConvBlock(nb_filter[0] * 2, nb_filter[0])
        self.conv2_2 = ConvBlock(nb_filter[1] * 2, nb_filter[1])
        self.conv3_2 = ConvBlock(nb_filter[2] * 2, nb_filter[2])
        self.conv4_2 = ConvBlock(nb_filter[3] * 2, nb_filter[3])

        # Final convolution layers (deep supervision outputs)
        self.final = nn.Conv2d(nb_filter[0], out_channels, kernel_size=1)  # Final output layer

    def forward(self, x):
        # Downsampling path
        x1_1 = self.conv1_1(x)
        x2_1 = self.conv2_1(F.max_pool2d(x1_1, 2))
        x3_1 = self.conv3_1(F.max_pool2d(x2_1, 2))
        x4_1 = self.conv4_1(F.max_pool2d(x3_1, 2))
        x5_1 = self.conv5_1(F.max_pool2d(x4_1, 2))

        # Upsampling path
        x4_2 = self.up4_2(x5_1)  # Upsample
        x4_2 = torch.cat([x4_2, x4_1], dim=1)  # Concatenate with downsampled x4
        x4_2 = self.conv4_2(x4_2)

        x3_2 = self.up3_2(x4_2)
        x3_2 = torch.cat([x3_2, x3_1], dim=1)
        x3_2 = self.conv3_2(x3_2)

        x2_2 = self.up2_2(x3_2)
        x2_2 = torch.cat([x2_2, x2_1], dim=1)
        x2_2 = self.conv2_2(x2_2)

        x1_2 = self.up1_2(x2_2)
        x1_2 = torch.cat([x1_2, x1_1], dim=1)
        x1_2 = self.conv1_2(x1_2)

        # Final output
        output = self.final(x1_2)
        return output

    def up_conv(self, in_channels, out_channels):
        """Upsample and return a ConvTranspose2d layer"""
        return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

# Instantiate the model
model = NestedUNet(in_channels=1, out_channels=1)


In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Dice coefficient metric
def dice_coeff(pred, target, smooth=1e-5):
    pred = torch.sigmoid(pred)  # Convert logits to probabilities
    pred = pred > 0.5  # Binarize predictions (threshold)
    intersection = (pred * target).sum()
    return (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

# Combined BCE + Dice loss with weighted Dice contribution
class BCEDiceLoss(nn.Module):
    def __init__(self, bce_weight=0.3, dice_weight=0.7):
        super(BCEDiceLoss, self).__init__()
        self.bce_loss = nn.BCEWithLogitsLoss()  # More stable than plain BCE
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight

    def forward(self, pred, target):
        bce_loss = self.bce_loss(pred, target)
        dice_loss = 1 - dice_coeff(pred, target)  # Dice loss term
        return self.bce_weight * bce_loss + self.dice_weight * dice_loss

# Optimizer with a higher learning rate
optimizer = optim.Adam(model.parameters(), lr=1e-3)  # Increased LR

# Training loop for one epoch with batch loss display
def train_one_epoch(model, dataloader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (images, masks) in enumerate(tqdm(dataloader, desc="Training Progress")):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # Show loss for each batch
        if batch_idx % 10 == 0:  # Display every 10 batches (or adjust as needed)
            print(f'Batch {batch_idx}, Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Validation loop
def validate(model, dataloader, loss_fn, device):
    model.eval()
    running_loss = 0.0
    dice_score = 0.0
    with torch.no_grad():
        for batch_idx, (images, masks) in enumerate(tqdm(dataloader, desc="Validation Progress")):
            images, masks = images.to(device), masks.to(device)

            outputs = model(images)
            loss = loss_fn(outputs, masks)
            running_loss += loss.item() * images.size(0)

            dice_score += dice_coeff(outputs, masks).item() * images.size(0)

            # Optionally, show validation batch loss if needed
            if batch_idx % 100 == 0:
                print(f'Validation Batch {batch_idx}, Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    avg_dice = dice_score / len(dataloader.dataset)
    return epoch_loss, avg_dice

# Full training process
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')

    # Train for one epoch
    train_loss = train_one_epoch(model, train_loader, optimizer, BCEDiceLoss(), device)

    # Validate after the epoch
    val_loss, val_dice = validate(model, val_loader, BCEDiceLoss(), device)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Dice: {val_dice:.4f}')


Epoch 1/5


Training Progress:   0%|          | 2/2946 [00:00<09:01,  5.44it/s]

Batch 0, Loss: 0.0891


Training Progress:   0%|          | 12/2946 [00:02<09:09,  5.34it/s]

Batch 10, Loss: 0.0735


...

Batch 280, Loss: 0.7097


Training Progress:  10%|▉         | 292/2946 [00:56<06:52,  6.44it/s]

Batch 290, Loss: 0.0030


Training Progress:  10%|█         | 299/2946 [00:57<08:32,  5.17it/s]


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os

# Dice coefficient metric
def dice_coeff(pred, target, smooth=1e-5):
    pred = torch.sigmoid(pred)  # Convert logits to probabilities
    pred = pred > 0.5  # Binarize predictions (threshold)
    intersection = (pred * target).sum()
    return (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

# Combined BCE + Dice loss with weighted Dice contribution
class BCEDiceLoss(nn.Module):
    def __init__(self, bce_weight=0.3, dice_weight=0.7):
        super(BCEDiceLoss, self).__init__()
        self.bce_loss = nn.BCEWithLogitsLoss()  # More stable than plain BCE
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight

    def forward(self, pred, target):
        bce_loss = self.bce_loss(pred, target)
        dice_loss = 1 - dice_coeff(pred, target)  # Dice loss term
        return self.bce_weight * bce_loss + self.dice_weight * dice_loss

# Optimizer with a higher learning rate
optimizer = optim.Adam(model.parameters(), lr=1e-3)  # Increased LR

# Training loop for one epoch with batch loss display
def train_one_epoch(model, dataloader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (images, masks) in enumerate(tqdm(dataloader, desc="Training Progress")):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # Show loss for each batch
        if batch_idx % 10 == 0:  # Display every 10 batches (or adjust as needed)
            print(f'Batch {batch_idx}, Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Validation loop
def validate(model, dataloader, loss_fn, device):
    model.eval()
    running_loss = 0.0
    dice_score = 0.0
    with torch.no_grad():
        for batch_idx, (images, masks) in enumerate(tqdm(dataloader, desc="Validation Progress")):
            images, masks = images.to(device), masks.to(device)

            outputs = model(images)
            loss = loss_fn(outputs, masks)
            running_loss += loss.item() * images.size(0)

            dice_score += dice_coeff(outputs, masks).item() * images.size(0)

            # Optionally, show validation batch loss if needed
            if batch_idx % 100 == 0:
                print(f'Validation Batch {batch_idx}, Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    avg_dice = dice_score / len(dataloader.dataset)
    return epoch_loss, avg_dice

# Define the directory to save the model
save_dir = '/content/drive/MyDrive/nested_unet'
os.makedirs(save_dir, exist_ok=True)  # Create directory if it doesn't exist

# Full training process
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')

    # Train for one epoch
    train_loss = train_one_epoch(model, train_loader, optimizer, BCEDiceLoss(), device)

    # Validate after the epoch
    val_loss, val_dice = validate(model, val_loader, BCEDiceLoss(), device)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Dice: {val_dice:.4f}')

    # Save the model after each epoch
    model_save_path = os.path.join(save_dir, f'model_epoch_{epoch+1}.pth')
    torch.save(model.state_dict(), model_save_path)
    print(f'Model saved to {model_save_path}')


Epoch 1/5


Training Progress:   0%|          | 1/2946 [00:00<12:10,  4.03it/s]

Batch 0, Loss: 0.0021


Training Progress:   0%|          | 12/2946 [00:02<08:16,  5.91it/s]

Batch 10, Loss: 0.0021


...

Training Progress:  63%|██████▎   | 1852/2946 [05:59<03:57,  4.62it/s]

Batch 1850, Loss: 0.0027


Training Progress:  63%|██████▎   | 1862/2946 [06:01<03:15,  5.55it/s]

Batch 1860, Loss: 0.0031


Training Progress:  63%|██████▎   | 1863/2946 [06:01<03:45,  4.80it/s]

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class AttentionBlock(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionBlock, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

class AttentionUNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(AttentionUNet, self).__init__()
        nb_filter = [64, 128, 256, 512, 1024]

        # Downsampling path
        self.conv1 = ConvBlock(in_channels, nb_filter[0])
        self.conv2 = ConvBlock(nb_filter[0], nb_filter[1])
        self.conv3 = ConvBlock(nb_filter[1], nb_filter[2])
        self.conv4 = ConvBlock(nb_filter[2], nb_filter[3])
        self.conv5 = ConvBlock(nb_filter[3], nb_filter[4])

        # Upsampling path
        self.up5 = nn.ConvTranspose2d(nb_filter[4], nb_filter[3], kernel_size=2, stride=2)
        self.att5 = AttentionBlock(F_g=nb_filter[3], F_l=nb_filter[3], F_int=nb_filter[2])
        self.upconv5 = ConvBlock(nb_filter[4], nb_filter[3])

        self.up4 = nn.ConvTranspose2d(nb_filter[3], nb_filter[2], kernel_size=2, stride=2)
        self.att4 = AttentionBlock(F_g=nb_filter[2], F_l=nb_filter[2], F_int=nb_filter[1])
        self.upconv4 = ConvBlock(nb_filter[3], nb_filter[2])

        self.up3 = nn.ConvTranspose2d(nb_filter[2], nb_filter[1], kernel_size=2, stride=2)
        self.att3 = AttentionBlock(F_g=nb_filter[1], F_l=nb_filter[1], F_int=nb_filter[0])
        self.upconv3 = ConvBlock(nb_filter[2], nb_filter[1])

        self.up2 = nn.ConvTranspose2d(nb_filter[1], nb_filter[0], kernel_size=2, stride=2)
        self.att2 = AttentionBlock(F_g=nb_filter[0], F_l=nb_filter[0], F_int=32)
        self.upconv2 = ConvBlock(nb_filter[1], nb_filter[0])

        # Final output layer
        self.final = nn.Conv2d(nb_filter[0], out_channels, kernel_size=1)

    def forward(self, x):
        # Downsampling path
        x1 = self.conv1(x)
        x2 = self.conv2(F.max_pool2d(x1, 2))
        x3 = self.conv3(F.max_pool2d(x2, 2))
        x4 = self.conv4(F.max_pool2d(x3, 2))
        x5 = self.conv5(F.max_pool2d(x4, 2))

        # Upsampling path
        d5 = self.up5(x5)
        x4 = self.att5(d5, x4)
        d5 = torch.cat((x4, d5), dim=1)
        d5 = self.upconv5(d5)

        d4 = self.up4(d5)
        x3 = self.att4(d4, x3)
        d4 = torch.cat((x3, d4), dim=1)
        d4 = self.upconv4(d4)

        d3 = self.up3(d4)
        x2 = self.att3(d3, x2)
        d3 = torch.cat((x2, d3), dim=1)
        d3 = self.upconv3(d3)

        d2 = self.up2(d3)
        x1 = self.att2(d2, x1)
        d2 = torch.cat((x1, d2), dim=1)
        d2 = self.upconv2(d2)

        output = self.final(d2)
        return output

# Instantiate the model
attention_unet = AttentionUNet(in_channels=1, out_channels=1)

# Example input tensor
dummy_input = torch.randn(64, 1, 256, 256)  # Batch size 64, 1 channel, 256x256 image
output = attention_unet(dummy_input)
print(f"Output shape: {output.shape}")  # Should match the input shape (64, 1, 256, 256)


Output shape: torch.Size([64, 1, 256, 256])


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import os

# Dice coefficient metric
def dice_coeff(pred, target, smooth=1e-5):
    pred = torch.sigmoid(pred)  # Convert logits to probabilities
    pred = pred > 0.5  # Binarize predictions (threshold)
    intersection = (pred * target).sum()
    return (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)

# Combined BCE + Dice loss with weighted Dice contribution
class BCEDiceLoss(nn.Module):
    def __init__(self, bce_weight=0.3, dice_weight=0.7):
        super(BCEDiceLoss, self).__init__()
        self.bce_loss = nn.BCEWithLogitsLoss()  # More stable than plain BCE
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight

    def forward(self, pred, target):
        bce_loss = self.bce_loss(pred, target)
        dice_loss = 1 - dice_coeff(pred, target)  # Dice loss term
        return self.bce_weight * bce_loss + self.dice_weight * dice_loss

# Instantiate the Attention U-Net model
model = AttentionUNet(in_channels=1, out_channels=1)

# Define the optimizer with a higher learning rate
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Define the directory to save the model
save_dir = '/content/drive/MyDrive/attention_unet'
os.makedirs(save_dir, exist_ok=True)  # Create directory if it doesn't exist

# Training loop for one epoch with batch loss display
def train_one_epoch(model, dataloader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (images, masks) in enumerate(tqdm(dataloader, desc="Training Progress")):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # Show loss for each batch
        if batch_idx % 10 == 0:  # Display every 10 batches (or adjust as needed)
            print(f'Batch {batch_idx}, Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Validation loop
def validate(model, dataloader, loss_fn, device):
    model.eval()
    running_loss = 0.0
    dice_score = 0.0
    with torch.no_grad():
        for batch_idx, (images, masks) in enumerate(tqdm(dataloader, desc="Validation Progress")):
            images, masks = images.to(device), masks.to(device)

            outputs = model(images)
            loss = loss_fn(outputs, masks)
            running_loss += loss.item() * images.size(0)

            dice_score += dice_coeff(outputs, masks).item() * images.size(0)

            # Optionally, show validation batch loss if needed
            if batch_idx % 100 == 0:
                print(f'Validation Batch {batch_idx}, Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(dataloader.dataset)
    avg_dice = dice_score / len(dataloader.dataset)
    return epoch_loss, avg_dice

# Full training process
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')

    # Train for one epoch
    train_loss = train_one_epoch(model, train_loader, optimizer, BCEDiceLoss(), device)

    # Validate after the epoch
    val_loss, val_dice = validate(model, val_loader, BCEDiceLoss(), device)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Dice: {val_dice:.4f}')

    # Save the model after each epoch
    model_save_path = os.path.join(save_dir, f'model_epoch_{epoch+1}.pth')
    torch.save(model.state_dict(), model_save_path)
    print(f'Model saved to {model_save_path}')


Epoch 1/1


100%|██████████| 197/197 [25:18<00:00,  7.71s/it]


Training Loss: 1.4187 | Training Dice: 0.0218


KeyboardInterrupt: 